# Uebung 2

## Aufgabe 1

a) Generieren Sie eine Liste von allen Download-URLs für 10cm SWISSIMAGE Bilder **im Kanton Bern**. (nur die Liste, kein Download!)<br/>
(für die Abfrage soll das generalisierte Polygon des Kanton Bern vom BFS verwendet werden)

b) Wieviele Bilder sind das ?

c) Wieviel Disk Speicher würde bei einem Download aller Files benötigt ?


In [1]:
import geopandas as gpd
import shapely
import geopandas_stac as stac
import urllib.request


stac.getCollectionList(cache=True)
df = stac.getAssets("ch.swisstopo.swissimage-dop10")
df_swissimage_10cm = df.query('gsd == "0.1"')

total_bilder = len(df_swissimage_10cm)

kantone = gpd.read_file("daten/gemeindegrenzen/ggg_2021-LV95/shp/g1k21.shp", encoding="utf-8")
kantone = kantone.to_crs("EPSG:4326")

# Bern:
poly_gpd = kantone.query('KTNR == 2')

# Spatial join:
df_swissimage_bern = gpd.sjoin(df_swissimage_10cm, poly_gpd, op='intersects')
urls = stac.getUrlList(df_swissimage_bern)

In [2]:
print(len(urls), "Bilder von", total_bilder)

6563 Bilder von 49488


In [3]:
total = 0
cnt = 0
for url in urls[0:50]:
    file = urllib.request.urlopen(url)
    total = total + file.length
    cnt+=1
    print(f"\r {cnt}   ", end="")

ca_datenmenge = total/50 * len(urls)

 50   

In [4]:
print(ca_datenmenge/1024/1024, "MB")
print(ca_datenmenge/1024/1024/1024, "GB")

435147.5141500664 MB
424.94874428717424 GB


## Aufgabe 2

Bestimmen Sie die URL der swissALTI3D 0.5cm an einer beliebigen Position (WGS84) in der Schweiz. (nur URL, kein Download)

In [5]:
import geopandas as gpd
import shapely
import geopandas_stac as stac


def getSwissalti3d_50cm_pt(lng, lat):
    df = stac.getAssets("ch.swisstopo.swissalti3d")

    df_geotiff = df.query('type == "image/tiff; application=geotiff; profile=cloud-optimized"')
    df_geotiff = df_geotiff.query("gsd == 0.5")

    point = shapely.geometry.Point(lng, lat)
    dfpoint = gpd.GeoDataFrame(geometry=gpd.GeoSeries(point, crs="epsg:4326"))

    df_result = gpd.sjoin(df_geotiff, dfpoint, op='contains')

    return stac.getUrlList(df_result)


In [6]:
result = getSwissalti3d_50cm_pt(8.00848562097163, 46.56820123946869)
if len(result)>0:
    print(result[0])
else:
    print("kein Datensatz gefunden!")

https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2643-1157/swissalti3d_2019_2643-1157_0.5_2056_5728.tif
